In [2]:
import pandas as pd
import numpy as np
import prettytable
import csv, sqlite3
%load_ext sql
%sql sqlite:///my_data1.db

In [3]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
#Load the data to be used into a dataframe

url = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df = pd.read_csv(url)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               101 non-null    object
 1   Time (UTC)         101 non-null    object
 2   Booster_Version    101 non-null    object
 3   Launch_Site        101 non-null    object
 4   Payload            101 non-null    object
 5   PAYLOAD_MASS__KG_  101 non-null    int64 
 6   Orbit              101 non-null    object
 7   Customer           101 non-null    object
 8   Mission_Outcome    101 non-null    object
 9   Landing_Outcome    101 non-null    object
dtypes: int64(1), object(9)
memory usage: 8.0+ KB


In [5]:
#Load the dataset into an SQL table

df.to_sql("SPACEXTBL", con, if_exists = 'replace', index = False, method = "multi")

101

In [6]:
#Code to remove blank rows from the table

#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

#### Task 1: Display the names of the unique launch sites in the space mission

In [51]:
%%sql

SELECT DISTINCT "Launch_Site"
FROM "SPACEXTBL"

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### Task 2: Display 5 records where launch sites begin with the string 'CCA'

In [52]:
%%sql

SELECT *
FROM 'SPACEXTBL'
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### Task 3: Display the total payload mass carried by boosters launched by NASA (CRS)

In [7]:
%%sql

SELECT SUM("PAYLOAD_MASS__KG_")
AS "Total Mass (kg)"
FROM "SPACEXTBL"
WHERE "Customer" = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


Total Mass (kg)
45596


#### Task 4: Display average payload mass carried by booster version F9 v1.1

In [8]:
%%sql

SELECT AVG("PAYLOAD_MASS__KG_")
AS "Average Payload Mass (kg)"
FROM "SPACEXTBL"
WHERE "Booster_Version" = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


Average Payload Mass (kg)
2928.4


#### Task 5: List the date when the first succesful landing outcome in ground pad was acheived.

In [64]:
%%sql

SELECT MIN("Date")
AS "First successful ground pad landing"
FROM "SPACEXTBL"
WHERE "Landing_Outcome" = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


First successful ground pad landing
2015-12-22


#### Task 6: List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [9]:
%%sql

SELECT "Booster_Version", "PAYLOAD_MASS__KG_"
FROM "SPACEXTBL"
WHERE "Landing_Outcome" = 'Success (drone ship)'
AND "PAYLOAD_MASS__KG_" 
BETWEEN 4000 AND 6000

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


#### Task 7: List the total number of successful and failure mission outcomes

In [95]:
%%sql

SELECT 

    SUM(CASE WHEN "Mission_Outcome" 
        LIKE '%Success%' 
        THEN 1 
        ELSE 0 END) 
        AS "Number of successful missions",

    SUM(CASE WHEN "Mission_Outcome" 
        LIKE '%Failure%' 
        THEN 1 
        ELSE 0 END) 
        AS "Number of unsuccessful missions"

FROM "SPACEXTBL"

 * sqlite:///my_data1.db
Done.


Number of successful missions,Number of unsuccessful missions
100,1


#### Task 8: List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.

In [11]:
%%sql

SELECT DISTINCT "Booster_Version", "PAYLOAD_MASS__KG_"
FROM "SPACEXTBL"
WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM "SPACEXTBL")
ORDER BY "Booster_Version"

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


#### Task 9: List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [12]:
%%sql

SELECT "Date", "Landing_Outcome", "Booster_Version", "Launch_Site"
FROM "SPACEXTBL"
WHERE "Landing_Outcome" LIKE "Failure (drone ship)"
AND substr(Date, 0,5) = '2015'

 * sqlite:///my_data1.db
Done.


Date,Landing_Outcome,Booster_Version,Launch_Site
2015-01-10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
2015-04-14,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


#### Task 10: Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [110]:
%%sql

SELECT DISTINCT "Landing_Outcome", COUNT("Landing_Outcome") AS 'Count', "Date"
FROM "SPACEXTBL"
WHERE "Date" 
BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY "Count" DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count,Date
No attempt,10,2012-05-22
Success (drone ship),5,2016-04-08
Failure (drone ship),5,2015-01-10
Success (ground pad),3,2015-12-22
Controlled (ocean),3,2014-04-18
Uncontrolled (ocean),2,2013-09-29
Failure (parachute),2,2010-06-04
Precluded (drone ship),1,2015-06-28
